# This notebook perform fault slip analysis (FSA) based on principal stresses from CMG geomechanical simulations.

# 1. Extract grid coordinates and fault id from Petrel exported files. Only need to do once for a 3D grid in Petrel. Save it as numpy array for later use.

# 2. Extract principla stresses (STRESMXP, STRESMNP, STRESINT) from CMG simulation results (gmch.sr3 -> rwo -> numpy).

# 3. Peform fault slip analysis

## perform fault slip analysis on each case

In [6]:
import numpy as np
coor_fault = np.load('data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy')
coor_fault[:,:,:,3].shape

(107, 117, 5)

In [ ]:
import pandas as pd
import numpy as np
from fault_slip_analysis import FSA_stress_based

n_cases = 2
n_faults = 12
fault_info = pd.read_csv('data/raw/fault_strike_dip.csv')
coor_fault = np.load('data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy')
FSA_by_fault = np.full((107,117,5,6), np.nan)
for i in range(n_cases):
    print(f"Processing case{i+1}...")
    for j in range(n_faults):
        row = fault_info.loc[fault_info['fault_id'] == j].iloc[0]
        fault_slip = FSA_stress_based(
            stress_folder_path = "data/250819_stresses",
            parameter_file_path = "data/params_responses/250819_CMG_parameters.csv",
            fault_cell_file_path = 'data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy',
            # save_folder_path = "data/250819_FSA_stress",
            case_name = f"case{i+1}", 
            fault_id = j,
            fault_strike = row['fault_strike_deg'],
            fault_dip = row['dip_angle_deg']
            )

        # save analysis to the specific fault 
        fault_id_mask = (coor_fault[:,:,:,3] == j)
        FSA_by_fault[fault_id_mask] = fault_slip[fault_id_mask]

    # save results for one case
    np.save(f'data/250819_FSA/case{i+1}_FSA.npy', FSA_by_fault)

Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case1...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...
Processing case2...


In [4]:
FSA_by_fault.shape

(107, 117, 5, 6)

## calculate fault slip percentage on each case

In [2]:
import numpy as np

n_cases = 2
n_time = 6
fault_slip_mean = np.zeros((n_cases,n_time))
for case_num in range(1, n_cases+1):
    fault_slip = np.load(f"results/fault_slip_gmc/case{case_num}_fault_slip.npy")
    fault_slip_mean[case_num-1,:] = np.mean(fault_slip,axis=(0,1,2))

# save as csv
np.savetxt("results/fault_slip_gmc.csv",fault_slip_mean,delimiter=",",fmt="%.4f")
# save for DGSA responses
# year = 2050; year_list = [2030, 2040, 2050, 2060, 2550, 3050]
# np.savetxt("results/250819_CMG_responses2.csv",fault_slip_mean[:,year_list.index(year)],delimiter=",",fmt="%.4f")

## calculate fault slip percentage on a specific fault at a specific time

In [3]:
import numpy as np
# parameters
fault_id = 0; year = 2050; year_list = [2030, 2040, 2050, 2060, 2550, 3050]
# load data
coor_fault = np.load('results/JD_Sula_2025_flow_coor&fault_reservoir.npy')
fault_slip = np.load('results/fault_slip/case1_fault_slip.npy')

# Filter data for the specific fault_id
mask = (coor_fault[:,:,:,3] == fault_id)
fault_slip_val = fault_slip[:,:,:,year_list.index(year)][mask]
print(np.mean(fault_slip_val))

0.44986449864498645
